Feed forward NN using PyTorch

In [171]:
import torchvision
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, ToPILImage, Lambda

In [172]:
transforms = Compose([
  # transform image to tensor
  ToTensor(),
  # transform size of tensor from 1*28*28 to 784
  Lambda(lambda i: i.view(784))
])

train_data = MNIST(root='./', download=True, train=True, transform=transforms)
test_data = MNIST(root='./', download=True, train=False, transform=transforms)

In [173]:
def to_t(tensor):
  return tensor.to(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

In [174]:
from torch import nn, optim

# MNIST digit classification model
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    # MLP. inear layers with ReLU activation functions. Cross entropy loss function and SGD optimizer.
    self.layers = nn.Sequential(
        nn.Linear(784,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )
    self.loss = nn.CrossEntropyLoss()
    self.optimizer = optim.SGD(self.parameters(), lr=0.1)
    self.to(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
    
  def forward(self, x):
    # complete forward a
    return self.layers(x)

  def predict(self, x):
    # don't calucalte gradients for test data
    with torch.no_grad():
      # return predicted digit
      return torch.argmax(self.forward(x), axis=-1)

  def fit(self, x, y):
    # don't accumulate gradients
    self.optimizer.zero_grad()
    # complete forward pass
    y_prediction = self.forward(x)
    # compute model loss
    loss = self.loss(y_prediction, y)
    # compute gradient using back prop
    loss.backward()
    # update model parameters based on calculated gradient
    self.optimizer.step()
    # return model's loss
    return loss.item()


In [175]:
model = Model()

In [176]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [177]:
EPOCHS = 5
for i in range(EPOCHS):
  loss = 0
  for x, y in train_dataloader:
    # tensor device conversion to GPU if available
    x, y = to_t(x), to_t(y)
    # train
    loss += model.fit(x,y)
  loss /= len(train_dataloader)
  print(f'Epoch {i+1} - training loss: {loss:.4f}')

Epoch 1 - training loss: 0.3261
Epoch 2 - training loss: 0.1126
Epoch 3 - training loss: 0.0732
Epoch 4 - training loss: 0.0515
Epoch 5 - training loss: 0.0372


In [178]:
correct=0

for x, y in test_dataloader:
  x, y = to_t(x), to_t(y)
  y_prediciton = model.predict(x)
  correct += (y == y_prediciton).sum()

accuracy = correct/(len(test_dataloader)*BATCH_SIZE)

print('Final Results')
print('-------------')

print(f'Training Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


Final Results
-------------
Training Loss: 0.0372
Test Accuracy: 0.9614
